In [1]:
import torch
from torch import nn, optim
from jcopdl.callback import Callback, set_config
device = 'cpu'
import os
os.makedirs('models', exist_ok=True)

# dataset dan dataloader

In [2]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [3]:
bs = 128
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.ToTensor()

])


trainset = datasets.ImageFolder('data/train/',
                                transform=transform)
trainloader = DataLoader(trainset, batch_size=bs, shuffle=True, num_workers=4)


testset = datasets.ImageFolder('data/test/',
                               transform=transform)
testloader = DataLoader(testset, batch_size=bs, shuffle=False, num_workers=4)
trainset

Dataset ImageFolder
    Number of datapoints: 40000
    Root location: data/train/
    StandardTransform
Transform: Compose(
               Grayscale(num_output_channels=1)
               ToTensor()
           )

In [4]:
images, labels = next(iter(trainloader))
images.shape, labels.shape

(torch.Size([128, 1, 28, 28]), torch.Size([128]))

In [5]:
trainset.classes

['Ankle boot',
 'Bag',
 'Coat',
 'Dress',
 'Pullover',
 'Sandal',
 'Shirt',
 'Sneaker',
 'T-shirt or Top',
 'Trouser']

# arsitektur dan config

In [6]:
from torch.nn import BatchNorm1d

In [7]:
def linear_block(in_features, out_features, activation='relu', dropout=0.0):
    layers = [nn.Linear(in_features, out_features)]
    # if batch_norm:
    #     layers.append(BatchNorm1d(out_features))
    if activation == 'relu':
        layers.append(nn.ReLU())
    elif activation == 'sigmoid':
        layers.append(nn.Sigmoid())
    elif activation == 'tanh':
        layers.append(nn.Tanh())
    if dropout > 0.0:
        layers.append(nn.Dropout(dropout))
    return nn.Sequential(*layers)

In [8]:
class FashionClassifier(nn.Module):
    def __init__(self, input_size=1 * 28 * 28, output_size=10, dropout=0.2):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Flatten(),
            linear_block(input_size, 256, activation='relu', dropout=dropout),
            linear_block(256, 64,  activation='relu', dropout=dropout),
            linear_block(64, 32,  activation='relu', dropout=dropout),
            nn.Linear(32, output_size),
            nn.LogSoftmax()
            )
    def forward(self, x):
        return self.fc(x)

In [9]:
condfig = set_config({
    'input_size':1 * 28 * 28,
    'n2':256,
    'n3':128,
    'n4':64,
    'output_size':len(trainset.classes),
    'dropout':0.2})

In [10]:
model = FashionClassifier().to(device)
criterion = nn.NLLLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
callback = Callback(model=model, outdir='models')

# Train

In [11]:
def loop_fn(mode, dataset, dataloader, model, criterion, optimizer, device):   
    from tqdm.auto import tqdm                             
    if mode == 'train':
        model.train()
    elif mode == 'test':
        model.eval()
    cost = correct = 0
    for feature, target in tqdm(dataloader, desc=mode.title()):
        feature, target = feature.to(device), target.to(device)
        output = model(feature)
        loss = criterion(output, target)
        
        if mode == 'train':
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        
        cost += loss.item() * feature.shape[0]
        correct += (output.argmax(dim=1) == target).sum().item()
    cost = cost / len(dataset)
    accuracy = correct / len(dataset)
    return cost, accuracy

In [ ]:
while True:
    train_cost, train_acc = loop_fn('train', trainset, trainloader, model, criterion, optimizer, device)
    with torch.no_grad():
        test_cost, test_acc = loop_fn('test', testset, testloader, model, criterion, optimizer, device)

    callback.log(train_cost=train_cost, test_cost=test_cost, test_score=test_acc, train_score=train_acc)
    callback.save_checkpoint()
    callback.cost_runtime_plotting()
    callback.score_runtime_plotting()
    if callback.early_stopping(model, monitor='test_score', load_best_when_stop=True):
        print("Early stopping triggered.")
        callback.plot_cost()
        callback.plot_score()
        break
    




    print(f'Train Cost: {train_cost:.4f}, Test Cost: {test_cost:.4f}')

Train:   0%|          | 0/313 [00:04<?, ?it/s]

c:\Users\dsapu\Project\conda\condasystem\envs\jcopdl\lib\site-packages\torch\nn\modules\container.py:139: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Test:   0%|          | 0/157 [00:04<?, ?it/s]


Epoch     1
Train_cost  = 0.8828 | Test_cost  = 0.5092 | Train_score = 0.6878 | Test_score = 0.8159 |
Train Cost: 0.8828, Test Cost: 0.5092


Train:   0%|          | 0/313 [00:04<?, ?it/s]

...


Epoch     3
Train_cost  = 0.4689 | Test_cost  = 0.4074 | Train_score = 0.8367 | Test_score = 0.8524 |
Train Cost: 0.4689, Test Cost: 0.4074


Train:   0%|          | 0/313 [00:04<?, ?it/s]

Test:   0%|          | 0/157 [00:04<?, ?it/s]


Epoch     4
Train_cost  = 0.4315 | Test_cost  = 0.3673 | Train_score = 0.8477 | Test_score = 0.8680 |
Train Cost: 0.4315, Test Cost: 0.3673


Train:   0%|          | 0/313 [00:04<?, ?it/s]